In [ ]:
# from tslearn.clustering import TimeSeriesKMeans
# from tslearn.clustering import silhouette_score

from collections import Counter
import copy
import csv
import cv2
import glob
import matplotlib
matplotlib.rcParams.update({'font.size': 16})
matplotlib.rcParams['font.family']="Arial"
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import pandas as pd
import peakutils
from scipy.spatial.distance import cdist
import scipy.stats as stats
from skimage import exposure
from sklearn.cluster import KMeans
import sklearn.decomposition #for PCA
from sklearn.metrics import silhouette_samples#, silhouette_score
import sklearn.preprocessing
from tifffile import imread
import matplotlib.cm as cm
from matplotlib import colors
# from skimage.external import tifffile as tif
from tifffile import imread
from matplotlib.patches import Ellipse
from scipy.stats import wilcoxon
from scipy.stats import ranksums


# pca = sklearn.decomposition.PCA(n_components=2)

# perform_PCA = False


In [ ]:
def get_dff(df):
    trial_f = df.iloc[13:]
    trial_dff = (trial_f-trial_f[0:14].mean(axis=0))/trial_f[0:14].mean(axis=0)
    return trial_dff

def k_resp_splitter4(side_tot, trial_list, resp_trials, noresp_trials):
    ind_trial_length = int(side_tot.shape[0]/len(trial_list))
    resp_act = []
    noresp_act = []
    
    for i in range(len(trial_list)):
        trial = trial_list[i]
        if trial in resp_trials:
            trial_f = side_tot.iloc[((ind_trial_length*int(trial_list.index(trial)))+13):(ind_trial_length*(int(trial_list.index(trial))+1))].reset_index().drop('index', axis=1)
            trial_dff = (trial_f-trial_f[0:14].mean(axis=0))/trial_f[0:14].mean(axis=0)
            print('trial_dff shape is', trial_dff.shape)
            resp_act.append(trial_dff)

        elif trial in noresp_trials:
            trial_f = side_tot.iloc[((ind_trial_length*int(trial_list.index(trial)))+13):(ind_trial_length*(int(trial_list.index(trial))+1))].reset_index().drop('index', axis=1)
            trial_dff = (trial_f-trial_f[0:14].mean(axis=0))/trial_f[0:14].mean(axis=0)
            noresp_act.append(trial_dff)
            
        else:
            pass
    resp_arr = pd.concat(resp_act, axis=0, ignore_index=True)
    print('resp_arr.shape is ', resp_arr.shape)
    if noresp_trials == '0':
        noresp_arr = resp_arr[:ind_trial_length]
        print('noresp_arr.shape is ', noresp_arr.shape)
        
    else:
        noresp_arr = pd.concat(noresp_act, axis=0, ignore_index=True)
    return resp_arr, noresp_arr

def fiji_elbow4(fijiarr):
    X = fijiarr
    
    distortions = []
    K = range(1,10)
    for k in K:
        kmeanModel = KMeans(n_clusters=k).fit(X)
        kmeanModel.fit(X)
        distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / len(X))

    # Plot the elbow
#     ax=plt.subplot(111)
    fig, ax = plt.subplots()
    fig.set_dpi(300)
#     fig = plt.figure(figsize=(30, 25), dpi=300)
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=15)
    
#     xticks = [0.24,0.29, 0.34]
#     xlabels = ['0.25','0.30', '0.35']
# #     xticks = [0.4,0.6, 0.8, 1.0]
# #     xlabels = ['0.4','0.6', '0.8', '1.0']
#     ax.set_yticks(xticks)
#     ax.set_yticklabels(xlabels)
    
    plt.plot(K, distortions, 'bx-')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k', pad=10)
#     plt.ylim(0.1,1.4)
    plt.savefig('complete_elbow.png')
    plt.show()
    

    n_clusters = int(input("How many clusters for kMeans: "))
    
    return n_clusters

def fiji_kmeans4(n_clusters, fijiarr):
    
    n_clusters = n_clusters
    fijiarr = fijiarr
    
    kmeans = sklearn.cluster.KMeans(n_clusters)
    clustered_data = kmeans.fit(fijiarr)

    batch_labelled_total_list = kmeans.labels_
    
    return batch_labelled_total_list

def fiji_kmeansA(n_clusters, fijiarr):
    
    n_clusters = n_clusters
    fijiarr = fijiarr
    
    kmeans = sklearn.cluster.KMeans(n_clusters)
    clustered_data = kmeans.fit(fijiarr)

    batch_labelled_total_list = kmeans.labels_
    
    return batch_labelled_total_list, kmeans

def fiji_organize_roisA(batch_labelled_total_list, fijiarr):
    '''Used to split ROIs back into their original planes AND 
    into the new clusters'''
    labels_by_plane = []
    clustered_temporal_footprints = {}
    label_proportions = {}
    
    side_dff = fijiarr
#     print('fiji_organize_roisA',clustered_temporal_footprints.keys())
    
    for label, calcium_trace in zip(batch_labelled_total_list, side_dff):
#         calcium_trace = k_concat_splitterA(concat_trace, trials)
#         k_concat_splitter(side_tot, trial_list)
        if label not in clustered_temporal_footprints.keys():
            clustered_temporal_footprints[label] = []
            clustered_temporal_footprints[label].append(calcium_trace)
        else:
            clustered_temporal_footprints[label].append(calcium_trace)
    for cluster in clustered_temporal_footprints.keys():
        label_proportions[cluster] = int(round(10*(len(clustered_temporal_footprints[cluster])/len(batch_labelled_total_list))))
       
    if 0 in label_proportions.values():
        for i in label_proportions:
            if label_proportions[i] == 0:
                label_proportions[i] = 1
    return clustered_temporal_footprints, label_proportions

def k_concat_splitterA(side_tot, trial_list):
    ind_trial_length = int(side_tot.shape[0]/len(trial_list))
    print('ind_trial_length is', ind_trial_length)
    split_act = []

    for i in range(len(trial_list)):
        trial = trial_list[i]
        if trial in trial_list:
            split_act.append(side_tot[(ind_trial_length*int(trial_list.index(trial))):(ind_trial_length*(int(trial_list.index(trial))+1))].reshape(-1,1))#.reset_index().drop('index', axis=1))

    split_act = np.concatenate(split_act, axis=1)#, ignore_index=True)
    split_mean = np.mean(split_act, axis=1)#I think this is a very elegant solution, but causes problems down the line
    return split_act#~


def plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters):
    '''Plots a line graph of the average calcium activity of each cluster.'''
    trial_duration = plot_duration
    total_frames = plot_frames
    avg_file_csv = str(name_for_file)+'.csv'
    avg_file_plot_png = str(name_for_file)+'.png'
    trace_counter = 0
    mean_dict = {}
    
    for i in range(0,n_clusters):
#         print(type(clustered_temporal_footprints[i]))
        print(len(clustered_temporal_footprints[i]))
        split_act = np.asarray(clustered_temporal_footprints[i]).T
#         split_act = np.concatenate(clustered_temporal_footprints[0], axis=1)#, ignore_index=True)#~
#         split_act = clustered_temporal_footprints[i]
        
        mean_dict[i] = np.mean(split_act, axis=1)#~
        print('mean_dict[',i,'].shape is ', mean_dict[i].shape)

        ######mean_dict[i] = np.mean(clustered_temporal_footprints[i], axis=0)
        
    #saves csv of mean cluster activity
    cluster_averages = pd.DataFrame(mean_dict).T
#     print('The shape of ')
    cluster_averages.to_csv(("cluster_avg_traces"+avg_file_csv))
    
    
    colors = ['tomato', 'seagreen', 'deepskyblue', 'orange', 'skyblue', 'magenta']
    xVals = np.linspace(0, trial_duration, total_frames)
    legend_names = mean_dict.keys()
    
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=15)
    fig, ax = plt.subplots()
    fig.set_dpi(300)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    xticks = [-0.2,0, 0.2, 0.4, 0.6]
    xlabels = ['-0.2','0', '0.2', '0.4', '0.6']
#     xticks = [-0.15,0, 0.15, 0.3]
#     xlabels = ['-0.15','0', '0.15', '0.3']    
    
    ax.set_yticks(xticks)
    ax.set_yticklabels(xlabels, fontname="Arial")
    plt.title(name_for_file, pad=15)    
    plt.title(name_for_file, pad=15)
    plt.ylim(-0.2,0.7)
#     plt.ylim(-0.1,0.3)
    
    for i in (mean_dict.keys()):


        plt.plot(xVals, mean_dict[i], color=colors[i])


#         plt.ylim(-0.2,0.4)

    plt.savefig(("cluster_avg_traces_"+avg_file_plot_png))
    plt.close()
    return mean_dict

def fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial):
    '''Plots heatmap raster plots of different clusters.'''
#     Things to fix:
    subplot_height = sum(label_proportions.values())
    heat_file_plot_png = str(name_for_file)+'.png'

    subplot_start = 0
    fig = plt.figure(figsize=(30, 25), dpi=300)
    gs0 = gridspec.GridSpec(1,1, figure=fig)
    gs00 = gridspec.GridSpecFromSubplotSpec(subplot_height, 11, subplot_spec=gs0[0], wspace=0.0)
#     extend = [0,40.2,0,110] # sets size of the plots, not sure if I still use this. Will check after I solve the colour plotting issue
    split2 = []
    for i in range(n_clusters):
        if i in clustered_temporal_footprints.keys():
#             split_act1 = np.concatenate(clustered_temporal_footprints[i], axis=1)#~
            split_act1 = np.asarray(clustered_temporal_footprints[i]).T
            print('clustered_temporal_footprints.keys', str(i))
            print('split_act1 is', split_act1.shape)
        else:
            split_act1 = np.ones((frames_per_trial,1), np.int8)
        #!!#!##!##!!#!###!#!#!#!#!#!#!#
        
        
        split2.append(split_act1)
#         fig.add_subplot(gs00[(subplot_start):(subplot_start+label_proportions[i]),0:-2])
    split_act2 = np.concatenate(split2, axis=1)
#     norm = colors.Normalize()#vmin=split_act2.T.min(), vmax=split_act2.T.max())
    norm = colors.Normalize(vmin=split_act2.T.min(), vmax=0.8)
    plt.imshow(split_act2.T, cmap = 'viridis', aspect = 'auto', extent= [0, trial_duration, 0, split_act2.shape[1]], norm=norm)
#     plt.colorbar()
#         plt.imshow(split_act1.T, cmap = 'plasma', aspect = 'auto', extent= [0, trial_duration, 0, split_act1.shape[1]], norm=norm)
    ####this is the old stuff I can go back to
#         split_act1 = np.concatenate(clustered_temporal_footprints[i], axis=1)#~
#         fig.add_subplot(gs00[(subplot_start):(subplot_start+label_proportions[i]),0:-2])
#         plt.ylabel(('Cluster '+str(i)), fontsize=25)
# #         norm = colors.Normalize(vmin=0, vmax=1)
#         plt.imshow(split_act1.T, cmap = 'plasma', aspect = 'auto', extent= [0, trial_duration, 0, split_act1.shape[1]])
# #         plt.imshow(split_act1.T, cmap = 'plasma', aspect = 'auto', extent= [0, trial_duration, 0, split_act1.shape[1]], norm=norm)

        #subplot_start += label_proportions[i]!!!This gets kept and uncommented
#     fig.suptitle(str(video1_name[:-10]), fontsize=40)#"{}".format(video_name[:-4]), fontsize=40)
#     plt.show()
    plt.savefig(('vHeat_ClustersCleanNORMnosep'+heat_file_plot_png))
    plt.close()


def plot_clust_indandavg(k, clustered_temporal_footprints, name):
    #plots all trials in a cluster, comment back in to generate plots
    plot_name = name
#     for i in range(0,k):
# #         clust = np.concatenate(clustered_temporal_footprints[i], axis=1)
#         clust = np.asarray(clustered_temporal_footprints[i]).T
#         df = pd.DataFrame(clust)
#         dfmean = df.mean(axis=1)
#         dfstd = df.std(axis=1)
#         ax=plt.subplot(111)
#         plt.plot(df, c='silver')
#         plt.plot(dfmean, c='black')
#         ax.spines['right'].set_visible(False)
#         ax.spines['top'].set_visible(False)        
# #         plt.ylim(-1,3.5)#!#!
#         plt.ylim(-0.2,0.7)
#         plt.savefig(plot_name+str(i)+'ing&avg.png')
#         plt.clf()
        
#         r = range(0,df.shape[0])
#         fig, ax = plt.subplots()
#         ax.plot(dfmean)
#         ax.fill_between(r, dfmean+dfstd, dfmean-dfstd, alpha=0.2)
#         ax.spines['right'].set_visible(False)
#         ax.spines['top'].set_visible(False)  
#         plt.ylim(-0.2,0.7)#!#!
# #         plt.ylim(-1,3.5)#!#!
#         plt.savefig(plot_name+str(i)+'std.png')
#         plt.clf()
        

In [ ]:
def plot_concated_clusters(batch_side_total_list, side_dff, file_stem, side, n_clusters):
    side_arr = np.array(side_dff).T
    clustered_tot_footprints = {}
    label_proportions = {}
    for label, concat_trace in zip(batch_side_total_list, side_arr):
        if label not in clustered_tot_footprints.keys():
            clustered_tot_footprints[label] = []
            clustered_tot_footprints[label].append(concat_trace.reshape(-1,1))
        else:
            clustered_tot_footprints[label].append(concat_trace.reshape(-1,1))
            
#     return clustered_tot_footprints
    for cluster in clustered_tot_footprints.keys():
        label_proportions[cluster] = int(round(10*(len(clustered_tot_footprints[cluster])/len(batch_side_total_list))))
    
    for i in range(n_clusters):
        if i not in label_proportions.keys():
            label_proportions.update({i:1})
        
    print('label proportions are:')
    print(label_proportions.values())
    print(label_proportions.keys())
    if 0 in label_proportions.values():
        for i in label_proportions:
            if label_proportions[i] == 0:
                label_proportions[i] = 1
                
    total_frames = int(side_dff.shape[0])
    trial_duration = total_frames/2.73
    name_for_file = file_stem+side+'concatPlot'
#     plot_cluster_avgsONLY4(clustered_tot_footprints, plot_frames, plot_duration, name_for_file)
    
    avg_file_csv = str(name_for_file)+'.csv'
    avg_file_plot_png = str(name_for_file)+'.png'
    trace_counter = 0
    concat_mean_dict = {}
    jk = 0

    for i in clustered_tot_footprints:
        if i in clustered_tot_footprints.keys():
            split_act = np.concatenate(clustered_tot_footprints[i], axis=1)#, ignore_index=True)#~
        else:
            split_act = np.ones((side_dff.shape[0],1), np.int8)
        concat_mean_dict[i] = np.mean(split_act, axis=1)#~
        if jk == 0:
            jk+=1
#         ######mean_dict[i] = np.mean(clustered_temporal_footprints[i], axis=0)
        
#     saves csv of mean cluster activity
    cluster_averages = pd.DataFrame(concat_mean_dict).T
    cluster_averages.to_csv(("cluster_avg_traces"+avg_file_csv))
    
    
    colors = ['tomato', 'seagreen', 'deepskyblue', 'orange', 'skyblue', 'magenta']
    xVals = np.linspace(0, trial_duration, total_frames)
    legend_names = concat_mean_dict.keys()
    
    
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=15)
    fig, ax = plt.subplots()
    fig.set_dpi(300)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    xticks = [-0.2,0, 0.2, 0.4, 0.6]
    xlabels = ['-0.2','0', '0.2', '0.4', '0.6']
#     xticks = [-0.15,0, 0.15, 0.3]
#     xlabels = ['-0.15','0', '0.15', '0.3']     
    
    ax.set_yticks(xticks)
    ax.set_yticklabels(xlabels, fontname="Arial")
    plt.title(name_for_file, pad=15)    
    plt.ylim(-0.2,0.7)
#     plt.ylim(-0.1,0.3)
    
  
    
    for i in (concat_mean_dict.keys()):
#         plt.rc('xtick', labelsize=15)
#         plt.rc('ytick', labelsize=15)
#         fig, ax = plt.subplots()
#         fig.set_dpi(300)
#         ax.spines['right'].set_visible(False)
#         ax.spines['top'].set_visible(False)
#         xticks = [-0.2,0, 0.2, 0.4, 0.6]
#         xlabels = ['-0.2','0', '0.2', '0.4', '0.6']
#         ax.set_yticks(xticks)
#         ax.set_yticklabels(xlabels)
#         plt.title(name_for_file, pad=10)
        plt.plot(xVals, concat_mean_dict[i], color=colors[i])
        
#         plt.ylim(-0.2,0.7)
#         plt.ylim(-0.2,0.4)

    plt.savefig(("cluster_avg_traces_"+avg_file_plot_png))
    plt.close()

    subplot_height = sum(label_proportions.values())
    heat_file_plot_png = str(name_for_file)+'.png'

    subplot_start = 0
    fig = plt.figure(figsize=(30, 25), dpi=300)
    gs0 = gridspec.GridSpec(1,1, figure=fig)
    gs00 = gridspec.GridSpecFromSubplotSpec(subplot_height, 11, subplot_spec=gs0[0], wspace=0.0)
#     extend = [0,40.2,0,110] # sets size of the plots, not sure if I still use this. Will check after I solve the colour plotting issue
    for i in range(n_clusters):
        if i in clustered_tot_footprints.keys():
            split_act1 = np.concatenate(clustered_tot_footprints[i], axis=1)#, ignore_index=True)#~
        else:
            split_act1 = np.ones((side_dff.shape[0],1), np.int8)
#         split_act1 = np.concatenate(clustered_tot_footprints[i], axis=1)#~
        fig.add_subplot(gs00[(subplot_start):(subplot_start+label_proportions[i]),0:-2])
        plt.ylabel(('Cluster '+str(i)), fontsize=25)
        plt.imshow(split_act1.T, cmap = 'seismic', aspect = 'auto', extent= [0, trial_duration, 0, split_act1.shape[1]])
#         plt.colorbar()
#         plt.imshow(clustered_temporal_footprints[i], cmap = 'plasma', aspect = 'auto', extent= [0, trial_duration, 0, len(clustered_temporal_footprints[i])])        
        subplot_start += label_proportions[i]
#     fig.suptitle(str(video1_name[:-10]), fontsize=40)#"{}".format(video_name[:-4]), fontsize=40)
#     plt.show()
    plt.savefig(('vHeat_ClustersClean'+heat_file_plot_png))
    plt.close()


In [ ]:
def plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters):
    '''Plots a line graph of the average calcium activity of each cluster.'''
    trial_duration = plot_duration
    total_frames = plot_frames
    avg_file_csv = str(name_for_file)+'.csv'
    avg_file_plot_png = str(name_for_file)+'.png'
    trace_counter = 0
    mean_dict = {}
    sem_dict = {}

    for i in range(0,n_clusters):
        print(len(clustered_temporal_footprints[i]))
        split_act = np.asarray(clustered_temporal_footprints[i]).T
        
        mean_dict[i] = np.mean(split_act, axis=1)#~
        print(split_act.shape[1])
#         b=input('aw shit here we go again')
        sem_dict[i] = np.std(split_act, axis=1)#/math.sqrt(split_act.shape[1])

        ######mean_dict[i] = np.mean(clustered_temporal_footprints[i], axis=0)
        
    #saves csv of mean cluster activity
    cluster_averages = pd.DataFrame(mean_dict)#.T
#     print('The shape of ')
    cluster_averages.to_csv(("cluster_avg_traces"+avg_file_csv))
    
    
    colors = ['tomato', 'seagreen', 'deepskyblue', 'orange', 'skyblue', 'magenta']
    xVals = np.linspace(0, trial_duration, total_frames)
    legend_names = mean_dict.keys()
    
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=15)
    fig, ax = plt.subplots()
    fig.set_dpi(300)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    xticks = [-0.2,0, 0.2, 0.4, 0.6]
    xlabels = ['-0.2','0', '0.2', '0.4', '0.6']  
#     xticks = [-0.15,0, 0.15, 0.3]
#     xlabels = ['-0.15','0', '0.15', '0.3'] 
    ax.set_yticks(xticks)
    ax.set_yticklabels(xlabels, fontname="Arial")
    plt.title(name_for_file, pad=15)    
    plt.ylim(-0.2,0.7)
#     plt.ylim(-0.1,0.3)

    

    for i in (mean_dict.keys()):
#         plt.rc('xtick', labelsize=15)
#         plt.rc('ytick', labelsize=15)
#         fig, ax = plt.subplots()
#         fig.set_dpi(300)
        
#         xticks = [-0.2,0, 0.2, 0.4, 0.6]
#         xlabels = ['-0.2','0', '0.2', '0.4', '0.6']
#         ax.set_yticks(xticks)
#         ax.set_yticklabels(xlabels)
#         plt.title(name_for_file, pad=10)
        
        dfstd = mean_dict[i].std()#/math.sqrt(len(mean_dict[i]))

#         print(mean_dict[i].shape)
# #         b = input('here we go')
        ax.plot(xVals, mean_dict[i], color=colors[i])
        ax.fill_between(xVals, mean_dict[i]+sem_dict[i], mean_dict[i]-sem_dict[i], alpha=0.2, color=colors[i],edgecolor='None')
#         ax.spines['right'].set_visible(False)
#         ax.spines['top'].set_visible(False) 
#         plt.title(name_for_file, pad=10)
#         plt.ylim(-0.25,0.7)
# #         plt.ylim(-0.2,0.4)

    plt.savefig(("cluster_avg_traces_"+avg_file_plot_png))
    plt.close()
    return mean_dict

def remove_outliers(raw_df):
    pdstd = raw_df.std()
    pdmean = raw_df.mean()
    pddiff = raw_df-pdmean
    
    lim = pdstd*3
    bound = raw_df.where(abs(pddiff)<lim)
    t2=bound.dropna()
    return t2

def tenind_fun(df):#$%
    tenindtot = []
    for i in df.columns:
        act=pd.Series(df[i])
        mval = act.iloc[62:72].max()#$%^
        mvalind= (act==mval).idxmax()
        revmax =pd.Series(act[:mvalind+1].values[::-1])
        frommax = (revmax<(mval*0.1)).idxmax()

        tenind = mvalind-frommax
        tenindtot.append(tenind)

    return tenindtot

def percentind_fun(df, percent):#$%

    percentindtot = []
    for i in df.columns:
        act=pd.Series(df[i])
        mval = act.iloc[62:72].max()#$%^
        mvalind= (act==mval).idxmax()
#         (a==56).idxmax()
        revmax =pd.Series(act[:mvalind+1].values[::-1])
#         revmax =pd.Series(act[:act.idxmax()+1].values[::-1])
        frommax = (revmax<(mval*percent)).idxmax()
        percentind = mvalind-frommax
        percentindtot.append(percentind)

    return percentindtot

def complete_stats(df, fname):

    tot_dff=df
    fname=fname
    maxval = tot_dff.iloc[55:109,:].max()#new iloc for finding max values
    maxval =remove_outliers(maxval)
    
    maxind = (tot_dff == maxval).idxmax()
    
    truemaxtime = (maxind+13)/2.73
    
    baseval = abs(tot_dff.iloc[0:14,:].mean())
    endval = abs(tot_dff.iloc[-6:,:].mean())
    tenbaseval = baseval*1.1
    ninetymaxval = maxval*0.9
    tenmaxval = maxval*0.2
    totdecay = 100-((endval/maxval)*100)
    totdecay =remove_outliers(totdecay)
    
    tenind = pd.Series(percentind_fun(df, 0.1))
    ninetyind = pd.Series(percentind_fun(df, 0.9))
    
    risetime = (ninetyind-tenind)/2.73
    
    finaldf = pd.concat((baseval, maxval, maxind, truemaxtime, tenbaseval, tenmaxval, ninetymaxval, risetime, totdecay, ninetyind, tenind), axis=1)
    finaldf.columns = ['baseline', 'maxvalue', 'maxind', 'truemaxtime', '10%base', '10%max', '90%max', 'risetimemax', '%decay', 'ninetyind', 'tenind']
    finaldf.to_csv(fname+'stats.csv')
    return finaldf

In [ ]:
def apply_clust(file_stem, group, stain, kmeans, n_clusters, img_speed, left_tot_rois, right_tot_rois, batch_labelled_total_list):
    
    file_stem_end = '.csv'
    left_df = pd.read_csv(file_stem+group+'resp_'+stain+'_IpsAllStim'+file_stem_end, index_col=0)
    left_nodf = pd.read_csv(file_stem+group+'noresp_'+stain+'_IpsAllStim'+file_stem_end, index_col=0)
    right_df = pd.read_csv(file_stem+group+'resp_'+stain+'_ContAllStim'+file_stem_end, index_col=0)
    right_nodf = pd.read_csv(file_stem+group+'noresp_'+stain+'_ContAllStim'+file_stem_end, index_col=0)
    left_dff = get_dff(left_df)
    left_nodff = get_dff(left_nodf)
    right_dff = get_dff(right_df)
    right_nodff = get_dff(right_nodf)

    #get info for later functions
    left_tot_rois =left_tot_rois
    right_tot_rois = right_tot_rois
    frames_per_trial = int(left_dff.shape[0])
    trial_duration = frames_per_trial/2.73
    left_rois = left_df.shape[1]
    right_rois = right_df.shape[1]

    #combine sides
    tot_dff = pd.concat([left_dff, right_dff], axis=1)
    tot_nodff = pd.concat([left_nodff, right_nodff], axis=1)
    fijiarr = np.array(tot_dff).T
    fiji_noarr = np.array(tot_nodff).T

    batch_orig_list =batch_labelled_total_list
    batch_left = list(batch_orig_list[:left_rois])
    batch_right = list(batch_orig_list[left_tot_rois:left_tot_rois+right_rois])
    batch_labelled_total_list = batch_left+batch_right
    print('we are in ', str(group))
    print('The left and right dffs are of shape:', left_dff.shape, right_dff.shape)
    print('tot_dff and fijiarr are of shape:', tot_dff.shape, fijiarr.shape)
    print('batch_orig_list len is', len(batch_orig_list))
    print('batch_labelled_total_list len is', len(batch_labelled_total_list))
    print('left tot rois and left rois are', left_tot_rois, left_rois)
    print('right tot rois and right rois are', right_tot_rois, right_rois)

    stats_dict = {}
    
    for side in ('Ips', 'Cont'):
        if side == 'Ips':               
            name_for_file = file_stem+group+stain+side+'Resp'
            clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[0:left_rois],  fijiarr[0:left_rois])

            plot_frames = frames_per_trial
            plot_duration = plot_frames/img_speed
            mean_dict= plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
            fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
            plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
            
            clustdf = pd.DataFrame(clustered_temporal_footprints[1]).T
            print(type(clustdf))
            stat_key = side+'Resp'+str(1)
            stats_dict[stat_key] = complete_stats(clustdf, stat_key)#$%^
                    
            for i in range(0,n_clusters):
                clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                stat_key = side+'Resp'+str(i)
                stats_dict[stat_key] = complete_stats(clustdf, stat_key)#$%^

            name_for_file = file_stem+group+stain+side+'NoResp'
            clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[0:left_rois], fiji_noarr[0:left_rois])
            mean_dict = plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
#             mean_dict = plot_cluster_avgsONLY4(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file)
            fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
            plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
            for i in range(0,n_clusters):
                clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                stat_key = side+'NoResp'+str(i)
                stats_dict[stat_key] = complete_stats(clustdf, stat_key)

        elif side == 'Cont':
            name_for_file = file_stem+group+stain+side+'Resp'
            clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[left_rois:],  fijiarr[left_rois:])

            plot_frames = frames_per_trial
            plot_duration = plot_frames/img_speed
            mean_dict= plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
            fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
            plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
            for i in range(0,n_clusters):
                clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                stat_key = side+'Resp'+str(i)
                stats_dict[stat_key] = complete_stats(clustdf, stat_key)

            name_for_file = file_stem+group+stain+side+'NoResp'
            clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[left_rois:], fiji_noarr[left_rois:])
            mean_dict = plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
#             mean_dict = plot_cluster_avgsONLY4(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file)
            fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
            plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
            for i in range(0,n_clusters):
                clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                stat_key = side+'NoResp'+str(i)
                stats_dict[stat_key] = complete_stats(clustdf, stat_key)

    name_for_file = file_stem+group+stain+'combined'+'Resp'
    plot_combined_sides(batch_labelled_total_list, fijiarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)
    name_for_file = file_stem+group+stain+'combined'+'noResp'
    plot_combined_sides(batch_labelled_total_list, fiji_noarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)
    return stats_dict

In [ ]:
def plot_combined_sides(batch_labelled_total_list, fijiarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration):
    clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list,  fijiarr)
    plot_frames = frames_per_trial
    plot_duration = plot_frames/img_speed
    mean_dict= plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
    fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)

    
def combined_stats(batch_labelled_total_list, left_rois, fijiarr, n_clusters):
    stats_dict = {}
    for side in ('Ips', 'Cont'):
        if side == 'Ips':
            clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[0:left_rois],  fijiarr[0:left_rois])
            for i in range(0,n_clusters):
                clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                stat_key = side+'Resp'+str(i)
                stats_dict[stat_key] = complete_stats(clustdf, stat_key)
                
        elif side == 'Cont':
            clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[left_rois:],  fijiarr[left_rois:])
            for i in range(0,n_clusters):
                clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                stat_key = side+'Resp'+str(i)
                stats_dict[stat_key] = complete_stats(clustdf, stat_key)
        
    return stats_dict


In [ ]:
# %%timeit
def comp_analysis(fill):   
    file_stem = str(fill)
    
    img_speed = 2.73
    
    sep_type = 3
    
    if sep_type==1:
        #for R/NR
        #load rois and get df/f
        file_stem_end = '.csv'
        left_df = pd.read_csv(file_stem+'closeresp_rln_IpsAllStim'+file_stem_end, index_col=0)
        left_nodf = pd.read_csv(file_stem+'closenoresp_rln_IpsAllStim'+file_stem_end, index_col=0)
        right_df = pd.read_csv(file_stem+'closeresp_rln_ContAllStim'+file_stem_end, index_col=0)
        right_nodf = pd.read_csv(file_stem+'closenoresp_rln_ContAllStim'+file_stem_end, index_col=0)
        left_dff = get_dff(left_df)
        left_nodff = get_dff(left_nodf)
        right_dff = get_dff(right_df)
        right_nodff = get_dff(right_nodf)
        
        #get info for later functions
        frames_per_trial = int(left_dff.shape[0])
        trial_duration = frames_per_trial/2.73
        left_rois = left_df.shape[1]
        right_rois = right_df.shape[1]
        
        #combine sides
        tot_dff = pd.concat([left_dff, right_dff], axis=1)
        tot_nodff = pd.concat([left_nodff, right_nodff], axis=1)
        fijiarr = np.array(tot_dff).T
        fiji_noarr = np.array(tot_nodff).T
        
        n_clusters = fiji_elbow4(fijiarr)
        #can use the same batch_labelled_total_list because it is the same rois
        batch_labelled_total_list = fiji_kmeans4(n_clusters, fijiarr)
        
        stats_dict = {}
        
        for side in ('Ips', 'Cont'):
            if side == 'Ips':               
                name_for_file = file_stem+side+'Resp'
                clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[0:left_rois],  fijiarr[0:left_rois])

                plot_frames = frames_per_trial
                plot_duration = plot_frames/img_speed
                mean_dict= plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
                fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
                plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)#$%
                
                for i in range(0,n_clusters):
                    clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                    stat_key = side+'Resp'+str(i)
                    stats_dict[stat_key] = complete_stats(clustdf, stat_key)

                name_for_file = file_stem+side+'NoResp'
                clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[0:left_rois], fiji_noarr[0:left_rois])
                mean_dict = plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
#             mean_dict = plot_cluster_avgsONLY4(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file)
                fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
                plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
                for i in range(0,n_clusters):
                    clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                    stat_key = side+'NoResp'+str(i)
                    stats_dict[stat_key] = complete_stats(clustdf, stat_key)
        
#                 tot_left_dff = concat_dff4(trial_list, left_df)
#                 clustered_tot_footprints = plot_concated_clusters(batch_labelled_total_list[0:left_rois], tot_left_dff, file_stem, side, n_clusters)

            elif side == 'Cont':
                name_for_file = file_stem+side+'Resp'
                clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[left_rois:],  fijiarr[left_rois:])

                plot_frames = frames_per_trial
                plot_duration = plot_frames/img_speed
                mean_dict= plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
                fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
                plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
                for i in range(0,n_clusters):
                    clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                    stat_key = side+'Resp'+str(i)
                    stats_dict[stat_key] = complete_stats(clustdf, stat_key)

                name_for_file = file_stem+side+'NoResp'
                clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[left_rois:], fiji_noarr[left_rois:])
                mean_dict = plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
#             mean_dict = plot_cluster_avgsONLY4(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file)
                fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
                plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
                for i in range(0,n_clusters):
                    clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                    stat_key = side+'NoResp'+str(i)
                    stats_dict[stat_key] = complete_stats(clustdf, stat_key)
                    
        name_for_file = file_stem+'combined'+'Resp'
        plot_combined_sides(batch_labelled_total_list, fijiarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)
        name_for_file = file_stem+'combined'+'noResp'
        plot_combined_sides(batch_labelled_total_list, fiji_noarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)
        return 1,batch_labelled_total_list
                
    if sep_type==2:
        file_stem_end = '.csv'
        left_df = pd.read_csv(file_stem+'PAG6stim_rln_IpsAllStim'+file_stem_end, index_col=0)
        right_df = pd.read_csv(file_stem+'PAG6stim_rln_ContAllStim'+file_stem_end, index_col=0)
        
        left_dff = get_dff(left_df)
        right_dff = get_dff(right_df)
        
        #get info for later functions
        frames_per_trial = int(left_dff.shape[0])
        trial_duration = frames_per_trial/2.73
        plot_frames = frames_per_trial
        plot_duration = plot_frames/img_speed
        left_rois = left_df.shape[1]
        right_rois = right_df.shape[1]
        
        tot_dff = pd.concat([left_dff, right_dff], axis=1)
        fijiarr = np.array(tot_dff).T
        n_clusters = fiji_elbow4(fijiarr)
        print('n_cluster right after fiji_elbow is', n_clusters)
        batch_labelled_total_list = fiji_kmeans4(n_clusters, fijiarr)
        
        name_for_file = file_stem+'total'+'Resp'
        plot_combined_sides(batch_labelled_total_list, fijiarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)
        return 1,batch_labelled_total_list
        
    if sep_type==3:
        file_stem_end = '.csv'
#         Final_sens&beh_PAG6_rln_IpsAllStim
#         Final_sens&beh_PAG6_rln_IpsAllStim
        left_df = pd.read_csv(file_stem+'PAG6BehSensNov_rlnIpsAllStim'+file_stem_end, index_col=0)
        right_df = pd.read_csv(file_stem+'PAG6BehSensNov_rlnContAllStim'+file_stem_end, index_col=0)
        left_dff = get_dff(left_df)
        right_dff = get_dff(right_df)
        
        print('The left and right dfs are of shape:', left_df.shape, right_df.shape)
        print('The left and right dffs are of shape:', left_dff.shape, right_dff.shape)
        
        #get info for later functions
        frames_per_trial = int(left_dff.shape[0])
        trial_duration = frames_per_trial/2.73
        plot_frames = frames_per_trial
        plot_duration = plot_frames/img_speed
        left_rois = left_df.shape[1]
        print('left_rois are', left_rois)
        right_rois = right_df.shape[1]
        
        tot_dff = pd.concat([left_dff, right_dff], axis=1)
        fijiarr = np.array(tot_dff).T
        n_clusters = fiji_elbow4(fijiarr)
        print('n_cluster right after fiji_elbow is', n_clusters)
        batch_labelled_total_list, kmeans = fiji_kmeansA(n_clusters, fijiarr)
        
#         stats_dict=1
#         return stats_dict , batch_labelled_total_list
        
        print('The left and right dffs are of shape:', left_dff.shape, right_dff.shape)
        print('tot_dff and fijiarr are of shape:', tot_dff.shape, fijiarr.shape)
        print('batch_labelled_total_list len is', len(batch_labelled_total_list))
        
        name_for_file = file_stem+'total'+'Resp'
        plot_combined_sides(batch_labelled_total_list, fijiarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)

        
        name_for_file = file_stem+'TotIps'
        clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[:left_rois],  fijiarr[:left_rois])
        mean_dict= plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
        fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
#         plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
#         for i in range(0,n_clusters):
#             clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
#             stat_key = side+'Resp'+str(i)
#             stats_dict[stat_key] = complete_stats(clustdf, stat_key)

        name_for_file = file_stem+'TotCont'
        clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[left_rois:], fijiarr[left_rois:])
        mean_dict = plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
        fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
#         plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
#         for i in range(0,n_clusters):
#             clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
#             stat_key = side+'NoResp'+str(i)
#             stats_dict[stat_key] = complete_stats(clustdf, stat_key)
#         stats_dict=1
#         return stats_dict , batch_labelled_total_list
                    
        types = ['2025', '2535']
#         exps = ['rln', 'penk']
        all_stats = {}#$%^
      
        all_stats['combined']=combined_stats(batch_labelled_total_list, left_rois, fijiarr, n_clusters)
        print()
        for group in types: 
            stain = 'rln'
            group_stats =apply_clust(file_stem, group, stain, kmeans, n_clusters, img_speed, left_rois, right_rois, batch_labelled_total_list)
            all_stats[group] = group_stats
            print()
        return all_stats,batch_labelled_total_list 
    
stats_dict, batch_labelled_total_list = comp_analysis('Final_')


In [ ]:
renum1 = renum(batch_labelled_total_list, [1,2,0])
# renum1 = renum(batch_labelled_total_list, [2,0,1])

In [ ]:
hold0,hold1 = recolour('Final_', renum1)

In [ ]:

def renum(batch_total_list, act_seq):
    letlist = []
    cor_list = []
    for i in batch_total_list:
        if i == act_seq[0]:
            letlist.append('a')
        elif i == act_seq[1]:
            letlist.append('b')
        elif i== act_seq[2]:
            letlist.append('c')
    for i in letlist:
        if i =='a':
            cor_list.append(0)
        elif i=='b':
            cor_list.append(1)
        elif i=='c':
            cor_list.append(2)
        else:
            print('error')
    return cor_list
        
def recolour(fill, batch_rois):   
    file_stem = str(fill)
    batch_labelled_total_list = batch_rois
    
    img_speed = 2.73
    
    sep_type = 3
    
    if sep_type==1:
        #for R/NR
        #load rois and get df/f
        file_stem_end = '.csv'
        left_df = pd.read_csv(file_stem+'closeresp_rln_IpsAllStim'+file_stem_end, index_col=0)
        left_nodf = pd.read_csv(file_stem+'closenoresp_rln_IpsAllStim'+file_stem_end, index_col=0)
        right_df = pd.read_csv(file_stem+'closeresp_rln_ContAllStim'+file_stem_end, index_col=0)
        right_nodf = pd.read_csv(file_stem+'closenoresp_rln_ContAllStim'+file_stem_end, index_col=0)
        left_dff = get_dff(left_df)
        left_nodff = get_dff(left_nodf)
        right_dff = get_dff(right_df)
        right_nodff = get_dff(right_nodf)
        
        #get info for later functions
        frames_per_trial = int(left_dff.shape[0])
        trial_duration = frames_per_trial/2.73
        left_rois = left_df.shape[1]
        right_rois = right_df.shape[1]
        
        #combine sides
        tot_dff = pd.concat([left_dff, right_dff], axis=1)
        tot_nodff = pd.concat([left_nodff, right_nodff], axis=1)
        fijiarr = np.array(tot_dff).T
        fiji_noarr = np.array(tot_nodff).T
        
        n_clusters = fiji_elbow4(fijiarr)

        stats_dict = {}
        
        for side in ('Ips', 'Cont'):
            if side == 'Ips':               
                name_for_file = file_stem+side+'Resp'
                clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[0:left_rois],  fijiarr[0:left_rois])

                plot_frames = frames_per_trial
                plot_duration = plot_frames/img_speed
                mean_dict= plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
                fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
                plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
                for i in range(0,n_clusters):
                    clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                    stat_key = side+'Resp'+str(i)
                    stats_dict[stat_key] = complete_stats(clustdf, stat_key)

                name_for_file = file_stem+side+'NoResp'
                clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[0:left_rois], fiji_noarr[0:left_rois])
                mean_dict = plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
#             mean_dict = plot_cluster_avgsONLY4(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file)
                fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
                plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
                for i in range(0,n_clusters):
                    clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                    stat_key = side+'NoResp'+str(i)
                    stats_dict[stat_key] = complete_stats(clustdf, stat_key)

            elif side == 'Cont':
                name_for_file = file_stem+side+'Resp'
                clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[left_rois:],  fijiarr[left_rois:])

                plot_frames = frames_per_trial
                plot_duration = plot_frames/img_speed
                mean_dict= plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
                fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
                plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
                for i in range(0,n_clusters):
                    clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                    stat_key = side+'Resp'+str(i)
                    stats_dict[stat_key] = complete_stats(clustdf, stat_key)

                name_for_file = file_stem+side+'NoResp'
                clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[left_rois:], fiji_noarr[left_rois:])
                mean_dict = plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
#             mean_dict = plot_cluster_avgsONLY4(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file)
                fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
                plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
                for i in range(0,n_clusters):
                    clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
                    stat_key = side+'NoResp'+str(i)
                    stats_dict[stat_key] = complete_stats(clustdf, stat_key)
                    
        name_for_file = file_stem+'combined'+'Resp'
        plot_combined_sides(batch_labelled_total_list, fijiarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)
        name_for_file = file_stem+'combined'+'noResp'
        plot_combined_sides(batch_labelled_total_list, fiji_noarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)
        return 1,batch_labelled_total_list

    if sep_type==2:
        file_stem_end = '.csv'
        left_df = pd.read_csv(file_stem+'OMRstim_rln_IpsAllStim'+file_stem_end, index_col=0)
        right_df = pd.read_csv(file_stem+'OMRstim_rln_ContAllStim'+file_stem_end, index_col=0)
        left_dff = get_dff(left_df)
        right_dff = get_dff(right_df)
        
        #get info for later functions
        frames_per_trial = int(left_dff.shape[0])
        trial_duration = frames_per_trial/2.73
        plot_frames = frames_per_trial
        plot_duration = plot_frames/img_speed
        left_rois = left_df.shape[1]
        right_rois = right_df.shape[1]
        
        tot_dff = pd.concat([left_dff, right_dff], axis=1)
        fijiarr = np.array(tot_dff).T
        n_clusters = fiji_elbow4(fijiarr)
        print('n_cluster right after fiji_elbow is', n_clusters)
#         batch_labelled_total_list = fiji_kmeans4(n_clusters, fijiarr)
        
        name_for_file = file_stem+'total'+'Resp'
        plot_combined_sides(batch_labelled_total_list, fijiarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)
        return 1,batch_labelled_total_list
        
    if sep_type==3:
        file_stem_end = '.csv'
        left_df = pd.read_csv(file_stem+'PAG6BehSensNov_rlnIpsAllStim'+file_stem_end, index_col=0)
        right_df = pd.read_csv(file_stem+'PAG6BehSensNov_rlnContAllStim'+file_stem_end, index_col=0)
        left_dff = get_dff(left_df)
        right_dff = get_dff(right_df)
        
        print('The left and right dfs are of shape:', left_df.shape, right_df.shape)
        print('The left and right dffs are of shape:', left_dff.shape, right_dff.shape)
        
        #get info for later functions
        frames_per_trial = int(left_dff.shape[0])
        trial_duration = frames_per_trial/2.73
        plot_frames = frames_per_trial
        plot_duration = plot_frames/img_speed
        left_rois = left_df.shape[1]
        print('left_rois are', left_rois)
        right_rois = right_df.shape[1]
        
        tot_dff = pd.concat([left_dff, right_dff], axis=1)
        fijiarr = np.array(tot_dff).T
        n_clusters = fiji_elbow4(fijiarr)
        print('n_cluster right after fiji_elbow is', n_clusters)
        batch_labelled_total_list2, kmeans = fiji_kmeansA(n_clusters, fijiarr)
        print('The left and right dffs are of shape:', left_dff.shape, right_dff.shape)
        print('tot_dff and fijiarr are of shape:', tot_dff.shape, fijiarr.shape)
        print('batch_labelled_total_list len is', len(batch_labelled_total_list))
        
        name_for_file = file_stem+'total'+'Resp'
        plot_combined_sides(batch_labelled_total_list, fijiarr, frames_per_trial, img_speed, name_for_file, n_clusters, trial_duration)

        name_for_file = file_stem+'TotIps'
        clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[:left_rois],  fijiarr[:left_rois])
        mean_dict= plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
        fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
#         plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
#         for i in range(0,n_clusters):
#             clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
#             stat_key = side+'Resp'+str(i)
#             stats_dict[stat_key] = complete_stats(clustdf, stat_key)

        name_for_file = file_stem+'TotCont'
        clustered_temporal_footprints, label_proportions = fiji_organize_roisA(batch_labelled_total_list[left_rois:], fijiarr[left_rois:])
        mean_dict = plot_cluster_avgsONLYA(clustered_temporal_footprints, plot_frames, plot_duration, name_for_file, n_clusters)
        fiji_plot_heatmap_clustersA(label_proportions, clustered_temporal_footprints, n_clusters, trial_duration, name_for_file, frames_per_trial)
#         plot_clust_indandavg(n_clusters, clustered_temporal_footprints, name_for_file)
#         for i in range(0,n_clusters):
#             clustdf = pd.DataFrame(clustered_temporal_footprints[i]).T
#             stat_key = side+'NoResp'+str(i)
#             stats_dict[stat_key] = complete_stats(clustdf, stat_key)
        
        
        types = ['2535', '2025']
#         exps = ['rln', 'penk']
        all_stats = {}
    
        all_stats['combined']=combined_stats(batch_labelled_total_list, left_rois, fijiarr, n_clusters)
        for group in types: 
            stain = 'rln'
            group_stats =apply_clust(file_stem, group, stain, kmeans, n_clusters, img_speed, left_rois, right_rois, batch_labelled_total_list)
            all_stats[group] = group_stats
        return all_stats,batch_labelled_total_list 
    


### creates bar graph of counts of rois 

In [ ]:
def roi_count_plots(roidf):
    #plots counts of neurons used in the pag6
    key_val=0
    colourlist = ['deepskyblue', 'orchid', 'white']
    tick_place=[]
    tick_names = roidf.columns
    fig, ax = plt.subplots()
    for i in roidf.columns:
        if 'rln' in i:
            c = colourlist[0]
        elif 'penk' in i:
            c = colourlist[1]
        else:
            c = colourlist[2]
        rand_val = key_val+(np.random.rand(len(roidf[i]))/4)
        plt.scatter(rand_val, roidf[i], alpha=0.7, color=c, s=10)
        plt.bar(rand_val.mean(), roidf[i].mean(), width=0.7, alpha=0.4,color=c, yerr=roidf[i].std(), ecolor='dimgrey')
        tick_place.append(rand_val.mean())
        key_val+=1
    ax.set_xticks(tick_place)
    ax.set_xticklabels(tick_names)
    ax.tick_params(axis='x', rotation=90)
    plt.savefig('GlutGad_coexpproportion_neurons.png')

roidf = pd.read_csv('GlutGad_coexpproportion_neurons.csv', index_col=0)
roidf.columns
roi_count_plots(roidf)

### does stats

In [ ]:
def stat_plots():
    maxs = []
    xvals = []
    key_val = 0
    for i in keys:
        max_val = stats_dict[i]['maxval']
    #     print(len(max_val))
        rand_val = key_val+(np.random.rand(len(max_val))/4)
        maxs.append(max_val)
        xvals.append(rand_val)
        key_val+=2
    
    val_mean = []
    val_std = []
    val_xs = []
    for i in range(len(maxs)):
        val_mean.append(maxs[i].mean())
        val_std.append(maxs[i].std())
        val_xs.append(xvals[i].mean())
    plt.clf

    fig, ax = plt.subplots()
    ax.set_xticks(val_xs)
    ax.set_xticklabels(keys)
    ax.tick_params(axis='x', rotation=90)


    for i in range(len(maxs)):
        ax.scatter(xvals[i],maxs[i], c='darkgrey')
    ax.scatter(val_xs, val_mean, marker='_', color='black')
    ax.errorbar(val_xs, val_mean, yerr=val_std, color='black', linestyle='')

In [ ]:
inds = ['baseline', 'maxvalue', 'maxind', 'truemaxtime', '10%base', '10%max', '90%max', 'risetimemax', '%decay']#      %decay 

In [ ]:
def org_stats_tot(stats_dict, indgroups, stat_choice, window, groupname):

    plotgroup_names = indgroups
#     print(type(plotgroup_names))
    k_groups = {}
  
    key_val =0 
    group = 0
    group_color = ['tomato', 'seagreen', 'deepskyblue']
    tick_place=[]
    tick_names = plotgroup_names
    fig, ax = plt.subplots()
    
    fig.set_dpi(300)
    
    for i in plotgroup_names:
        if i in stats_dict[window].keys():
            if 'No' in i and 'No' not in groupname:
                c = 'grey'
            else:
                c = group_color[int(i[-1])]
                
            if stat_choice =='maxvalue':
                plt.ylim(0,1.2)
            elif stat_choice =='risetimemax':
                plt.ylim(0,10)
            elif stat_choice =='%decay':
                plt.ylim(-5,105)
                
                
            newval=remove_outliers(stats_dict[window][i][stat_choice])# to remove outliers
            rand_val = key_val+(np.random.rand(len(newval))/4)
            tick_place.append(rand_val.mean())
            plt.scatter(rand_val, newval, alpha=0.7, color=c, s=10)# to remove outliers
            plt.bar(rand_val.mean(), newval.mean(), width=0.7, alpha=0.5, color=c)#, yerr=newval.std(), ecolor='black', elinewidth=4)
            plt.errorbar(rand_val.mean(), newval.mean(),yerr=newval.std(), ecolor='black')#, elinewidth=4)
            group+=1
#             key_val+=1

        else:
#             key_val+=0.5
            rand_val = key_val
            tick_place.append(rand_val)
#             plt.scatter(rand_val, 0.1, c='w')
            group+=1
        if group ==2:
            key_val+=0.5
        
        key_val+=0.9
#         if i not in stats_dict[window].keys():
#             key_val-=0.5
            
    print('tick places are:',tick_place)
    print('tick names are:', tick_names)
    ax.set_xticks(tick_place)
    ax.set_xticklabels(tick_names)
    ax.tick_params(axis='x', rotation=90)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    penkrln = 'rln'

    plt.title(window+'_'+penkrln+'_'+stat_choice, pad=10)
    filetitle = window+penkrln+stat_choice+groupname+'graph'
#     plt.show()
#     exportEmf(savePath, filetitle)
    plt.savefig('ipscont'+window+penkrln+stat_choice+groupname+'graph.png')

def plot_all_stats():
    window = ['combined','2535', '2025']
    sider = ['Ips','Cont']
    sidenr = ['IpsNo','ContNo']
    ks = ['2','1']
    tot_groups = {}
    
#     for i in window:
#         for j,k in zip(sider, sidenr):
#             group=[]
#             for q in ks:
#                 group.append(j+'Resp'+q)
#                 group.append(k+'Resp'+q)
# #                 if len(group)==2:
# #                     group.append(' ')
#             tot_groups[str(i+j)]=group

    #plot ips/cont
    for i in window:
        group=[]
        for q in ks:
            for j in sider:
                group.append(j+'Resp'+q)
#                 if len(group)==2:
#                     group.append(' ')
            tot_groups[i+j]=group
        
        group=[]
        for q in ks:
            for j in sidenr:
                group.append(j+'Resp'+q)
#                 if len(group)==2:
#                     group.append(' ')
            tot_groups[i+j]=group
                   
    stat_choices = ['maxvalue','%decay', 'risetimemax']
    mm=1
        
    for i in tot_groups:
        if '2535' in i:
            window = '2535'
        elif '2025' in i:
            window='2025'
        elif 'combined' in i:
            window='combined'
        for stat_choice in stat_choices:
#             print(tot_groups[i])#, stat_choice, window, i)
            org_stats_tot(stats_dict, tot_groups[i], stat_choice, window, i)
            print(mm)
#             mm+=1
            
    return tot_groups

stats_dict = hold0
tot_groups=plot_all_stats()



In [ ]:
def org_dif_wind_stats_tot(stats_dict, indgroups, stat_choice, groupname):

    plotgroup_names = indgroups
    
    k_groups = {}
    window_names = ['2025', '2535']
  
    key_val =0 
    group = 0
    group_color = ['tomato', 'seagreen', 'deepskyblue']
    tick_place=[]
    tick_names = ['22', '32', '21', '31']
    fig, ax = plt.subplots()
    
    fig.set_dpi(300)
    
    for i in plotgroup_names:
        
        for window in window_names:
        
            if i in stats_dict[window].keys():
                c = group_color[int(i[-1])]

                if stat_choice =='maxvalue':
                    plt.ylim(0,1.2)
                elif stat_choice =='risetimemax':
                    plt.ylim(0,10)
                elif stat_choice =='%decay':
                    plt.ylim(-5,100)


                newval=remove_outliers(stats_dict[window][i][stat_choice])# to remove outliers
                rand_val = key_val+(np.random.rand(len(newval))/4)
                tick_place.append(rand_val.mean())
                plt.scatter(rand_val, newval, alpha=0.7, color=c, s=10)# to remove outliers
                plt.bar(rand_val.mean(), newval.mean(), width=0.7, alpha=0.5, color=c)#, yerr=newval.std(), ecolor='black', elinewidth=4)
                plt.errorbar(rand_val.mean(), newval.mean(),yerr=newval.std(), ecolor='black')#, elinewidth=4)
                group+=1
    #             key_val+=1

            else:
    #             key_val+=0.5
                rand_val = key_val
                tick_place.append(rand_val)
    #             plt.scatter(rand_val, 0.1, c='w')
                group+=1
            if group ==2:
                key_val+=0.5

            key_val+=0.9
#         if i not in stats_dict[window].keys():
#             key_val-=0.5
            
    print('tick places are:',tick_place)
    print('tick names are:', tick_names)
    ax.set_xticks(tick_place)
    ax.set_xticklabels(tick_names)
    ax.tick_params(axis='x', rotation=90)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    penkrln = 'rln'

    plt.title(window+'_'+penkrln+'_'+stat_choice, pad=10)
    filetitle = window+penkrln+stat_choice+groupname+'graph'
#     plt.show()
#     exportEmf(savePath, filetitle)
    plt.savefig('window'+window+penkrln+stat_choice+groupname+'graph.png')

def plot_dif_wind_all_stats():
    window = ['2535', '2025']
    sides = ['Ips','Cont','IpsNo','ContNo']
    # sidenr = ['IpsNo','ContNo']
    ks = ['2','1']
    tot_groups = {}
    
    for side in sides:
        group=[]
        for k in ks:
            group.append(str(side)+'Resp'+str(k))
        tot_groups[str(side)]=group

        
    stat_choices = ['maxvalue','%decay', 'risetimemax']

    for i in tot_groups.keys():
        groupen = []
        for j in tot_groups[i]:
            groupen.append(j)
        for choice in stat_choices:
            org_dif_wind_stats_tot(stats_dict, groupen, choice, i)
    
plot_dif_wind_all_stats()



In [ ]:
plotgroup_names, tick_place, tick_names=org_stats(stats_dict, 'big', 3)

In [ ]:
def plot_all_stats():
    window = ['2535', 'close']
    sider = ['Ips','Cont']
    sidenr = ['IpsNo','ContNo']
    ks = ['1','2']
    tot_groups = {}   
    #plot r/nr
    
    for i in window:
        for j,k in zip(sider, sidenr):
            group=[]
            for q in ks:
                group.append(j+'Resp'+q)
                group.append(k+'Resp'+q)
                if len(group)==2:
                    group.append(' ')
            tot_groups[str(i+j)]=group

    #plot ips/cont
    for i in window:
        group=[]
        for q in ks:
            for j in sider:
                group.append(j+'Resp'+q)
                if len(group)==2:
                    group.append(' ')
            tot_groups[i+j]=group
        
        group=[]
        for q in ks:
            for j in sidenr:
                group.append(j+'Resp'+q)
                if len(group)==2:
                    group.append(' ')
            tot_groups[i+j]=group
        
#     stat_choices = ['maxvalue','%decay', 'risetimemax']
        
#     for i in tot_groups:
#         print(i)
#         if 'big' in i:
#             window = 'big'
#         else:
#             window='close'
# #         for stat_choice in stat_choices:
# #             org_stats_tot(stats_dict, tot_groups[i], stat_choice, window, i)          
            
    return tot_groups
tot_groups=plot_all_stats()

In [ ]:
# nt= {'window':0, 'stat_choice':0, 'ICvNRN':0, 'g1':0, 'g2':0, 'stat':0, 'p':0}
window = ['combined','2025', '2535']
sider = ['IpsResp','ContResp']
sidenr = ['IpsNoResp','ContNoResp']
ks = ['1','2']
conds = ['ic', 'rnr', 'kn']
stat_choices = ['maxvalue','%decay', 'risetimemax']
mdict={}
m=0
p1 = 0
p2 = 1
for wind in window:
    for stat_choice in stat_choices:
        #doIC first
        for cond in conds:
            if cond=='ic':
                for k in ks:
                    if k == '1':
                        newnum='2'
                    elif k == '2':
                        newnum='1'
                    else:
                        print('didnt work')
                    g1= stats_dict[wind][sider[0]+k][stat_choice]#.dropna()
                    g2 = stats_dict[wind][sider[1]+k][stat_choice]#.dropna()
                    #NN
                    g1name = sider[0]+newnum
                    g2name = sider[1]+newnum
                    g1num = g1.shape[0]
                    g2num=g2.shape[0]
                    stat, p = ranksums(g1,g2)
                    stat_test = 'rank sum'
                    if wind == 'combined':
                        wind_name = 'combined'
                    elif wind == '2025':
                        wind_name = 'pre-exp'
                    elif wind == '2535':
                        wind_name = 'post-exp'
                    mdict[m]=(wind_name, stat_choice, 'Ips/Cont', g1name, g1num, g2name, g2num, stat_test, stat, p)
                    m+=1
                    
                    if wind=='combined':
                        continue
                    if k == 1:
                        newnum=2
                    elif k == 2:
                        newnum=1
                        
                    g1= stats_dict[wind][sidenr[0]+k][stat_choice]#.dropna()
                    g2 = stats_dict[wind][sidenr[1]+k][stat_choice]#.dropna()
                    g1name = sidenr[0]+newnum
                    g2name = sidenr[1]+newnum
                    g1num = g1.shape[0]
                    g2num=g2.shape[0]
                    stat, p = ranksums(g1,g2)
                    stat_test = 'rank sum'
                    if wind == 'combined':
                        wind_name = 'combined'
                    elif wind == '2025':
                        wind_name = 'pre-exp'
                    elif wind == '2535':
                        wind_name = 'post-exp'
                    mdict[m]=(wind_name, stat_choice, 'Ips/Cont', g1name, g1num, g2name, g2num, stat_test, stat, p)
                    m+=1
                    
            elif cond=='rnr':
                for k in ks:
                    if k == '1':
                        newnum='2'
                    elif k == '2':
                        newnum='1'
                    else:
                        print('didnt work')
                    if wind=='combined':
                        continue
                    g1= stats_dict[wind][sider[0]+k][stat_choice]
                    g2 = stats_dict[wind][sidenr[0]+k][stat_choice]
                    g1name = sider[0]+newnum
                    g2name = sidenr[0]+newnum
                    g1num = g1.shape[0]
                    g2num=g2.shape[0]
                    stat, p = wilcoxon(g1, g2)
                    stat_test = 'signed rank'
                    if wind == 'combined':
                        wind_name = 'combined'
                    elif wind == '2025':
                        wind_name = 'pre-exp'
                    elif wind == '2535':
                        wind_name = 'post-exp'
                    mdict[m]=(wind_name, stat_choice, 'R/NR', g1name, g1num, g2name, g2num, stat_test, stat, p)
                    m+=1
                    
                    g1= stats_dict[wind][sider[1]+k][stat_choice]
                    g2 = stats_dict[wind][sidenr[1]+k][stat_choice]
                    g1name = sider[1]+newnum
                    g2name = sidenr[1]+newnum
                    g1num = g1.shape[0]
                    g2num=g2.shape[0]
                    stat, p = wilcoxon(g1, g2)
                    stat_test = 'signed rank'
                    if wind == 'combined':
                        wind_name = 'combined'
                    elif wind == '2025':
                        wind_name = 'pre-exp'
                    elif wind == '2535':
                        wind_name = 'post-exp'
                    mdict[m]=(wind_name, stat_choice, 'R/NR', g1name, g1num, g2name, g2num, stat_test, stat, p)
                    m+=1
#             if cond=='kn':
#                 for side in sider:
# #                     if wind =='combined':
# #                         print(side, stat_choice)
#                     g1=stats_dict[wind][side+'1'][stat_choice]
#                     g2=stats_dict[wind][side+'2'][stat_choice]
#                     g1name = side+'1'
#                     g2name = side+'2'
#                     stat, p = ranksums(g1,g2)
#                     mdict[m]=(wind, stat_choice, cond, g1name, g2name, stat, p)
#                     m+=1
#                 for side in sidenr:
#                     if wind=='combined':
#                         continue
#                     g1=stats_dict[wind][side+'1'][stat_choice]
#                     g2=stats_dict[wind][side+'2'][stat_choice]
#                     stat, p = ranksums(g1,g2)
#                     mdict[m]=(wind, stat_choice, cond, g1name, g2name, stat, p)
#                     m+=1

print(m)
newdf = pd.DataFrame(mdict).T
newdf.columns=['window', 'stat_choice', 'cond', 'group1', 'group1 # neurons', 'group2', 'group2 # neurons', 'statistical test', 'statistic', 'p value']
newdf.to_csv('22rlnallstatscomp.csv')


sides = ['IpsResp', 'ContResp']
rnr = ['r', 'nr']
windows = ['2025', '2535']
ks = ['1', '2']
stat_choices = ['maxvalue','%decay', 'risetimemax']
mdict={}
m=0

for choice in stat_choices:
    for side in sides:        
        for k in ks:
            if k == '1':
                newnum='2'
            elif k == '2':
                newnum='1'
            else:
                print('didnt work')
#             newval=remove_outliers(stats_dict[window][i][stat_choice])# to remove outliers
#             g1 = remove_outliers(stats_dict[windows[0]][side+k][choice])#2025
#             print(g1.shape)
#             g2 = remove_outliers(stats_dict[windows[1]][side+k][choice])#2535
#             print(g2.shape)
            g1 = stats_dict[windows[0]][side+k][choice]
            g1num = g1.shape[0]
            g2 = stats_dict[windows[1]][side+k][choice]
            g2num=g2.shape[0]
            stat, p = wilcoxon(g1, g2)
            g1name = side+'_pre-exp'+newnum
            g2name = side+'_post-exp'+newnum
            stat_test = 'signed rank'
            mdict[m]=(side, choice, k, g1name, g1num, g2name, g2num, stat_test,  stat, p)
            m+=1
#             print(side+windows[0]+k+choice, side+windows[1]+k+choice)
#     print()

newdf = pd.DataFrame(mdict).T
newdf.columns=['window', 'stat_choice', 'cond', 'group1', 'group1 # neurons', 'group2', 'group2 # neurons', 'statistical test', 'statistic', 'p value']
newdf.to_csv('22rlnwindstatscomp.csv')